In [ ]:
import json

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('drive/gdrive/MyDrive/training_set.json')

In [ ]:
with open('training_set.json','rb') as f:
     squad = json.load(f)

UNDERSTADING THE  DATA STRUCTURE AND TYPE

In [ ]:
type(squad)

In [ ]:
squad.keys()

In [ ]:
squad

In [ ]:
type(squad['data'])

In [ ]:
print(len(squad['data']))

In [ ]:
squad['data']

In [ ]:
type(squad['data'][0])

In [ ]:
squad['data'][0].keys()

In [ ]:
type(squad['data'][1]['title'][0])

In [ ]:
type(squad['data'][0]['paragraphs'][0])

In [ ]:
squad['data'][0]['paragraphs'][0].keys()

In [ ]:
type(squad['data'][0]['paragraphs'][0]['context'][0])

In [ ]:
type(squad['data'][0]['paragraphs'][0]['qas'][0])

In [ ]:
squad['data'][0]['paragraphs'][0]['qas'][0].keys()

In [ ]:
type(squad['data'][0]['paragraphs'][0]['qas'][0] ['answers'][0])

In [ ]:
squad['data'][0]['paragraphs'][0]['qas'][0] ['answers'][0].keys()

In [ ]:
type(squad['data'][0]['paragraphs'][0]['qas'][0] ['id'][0])

In [ ]:
type(squad['data'][0]['paragraphs'][0]['qas'][0] ['question'][0])

CONSOLIDATION OF DATA TYPE

# POINT 1 : 'squad' is a complicated dictionary with two keys 'data' and 'version'. 

# POINT 2 : With respect to our work at hand, the value of the key'version' need not play any role at all. 

# POINT 3:  All expected analysis, will spin around the value corresponding to the 'data' key.

# POINT 4: The value corresponding to 'data' key is itself a list comprising 442 elements. 
#          Since we need to split the given data set into training and validation, it is at the 'data' key level we possibly
#          have to do the splitting.

# POINT 5: The 'data', which is of type list, the elements of that list is again a dictionary having two keys 
#          'title' and 'paragraphs'.

# POINT 6: 'title' is of type string whereas'paragraphs' is again of type dictionary with two keys 'context' and 'qas'.

# POINT 7:  'context' is of type strings and 'qas' is again a dictionary.

# POINT 8: The dictionary 'qas' contains three keys namely, 'answers','question' and 'id'.'id' and 'questions' are strings 
#          whereas 'answers' is again another dictionary with keys 'answer_start' and 'text'.

# POINT 9 : Also note, in the original data set we do not have any answer_end index as would be required as we use BERT or variants for building the Q/A application


Extracting the Contexts, Questions and Answers

In [ ]:
# Refer : https://huggingface.co/transformers/v4.0.1/custom_datasets.html

import json
from pathlib import Path

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('training_set.json')

In [ ]:
for group in squad['data']:
    print(group['title'])

Printing the Contexts, Questions and Answers

In [ ]:
# Note the repeatation of Context in the contexts list

train_contexts[:10]

In [ ]:
train_answers[:10]

In [ ]:
train_questions[:10]

In [ ]:
len(train_contexts)

In [ ]:
len(train_questions)

In [ ]:
len(train_answers)

Since the length of all the 3 lists is same, this means for each question associated with one context, there is ONLY one answer but the output of the code train_context[:25] reveals that for each context there can be several associated questions.

Computing the end index (To understand end of answers)

In [ ]:
# Refer : https://huggingface.co/transformers/v4.0.1/custom_datasets.html
 
def add_end_index(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_index = start_idx + len(gold_text)
        answer['answer_end'] = end_index

In [ ]:
add_end_index(train_answers,train_contexts)

In [ ]:
print(train_answers[:10])

Installing Transformers

In [ ]:
! pip install transformers

Performing Encoding

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_contexts,train_questions,padding=True,truncation=True)

The context-question pairs are now being represented as Encoding objects. These objects concatenate each corresponding context and question strings. Each context and question pair is separated with a [SEP] token.
This concatenated version is stored within the 'input_ids' attribute of the Encoding object.

Checking the encoded output

In [ ]:
train_encodings['input_ids'][0]

Checking the decoded output

In [ ]:
#Refer : https://huggingface.co/transformers/v4.0.1/custom_datasets.html

tokenizer.decode(train_encodings['input_ids'][0])

In [ ]:
tokenizer.decode(train_encodings['input_ids'][5])

ADD TOKEN POSITIONS

In [ ]:
type(train_encodings)

In [ ]:
type(train_encodings['input_ids'])

In [ ]:
type(train_encodings['attention_mask'])

In [ ]:
train_encodings.keys()

In [ ]:
train_encodings['attention_mask'][0]

The tokenization done prior, takes care of the Context and Question pair but nothing has been done as yet for the start and end positions of the answers for each question associated with a certain context. To achieve this, the below function has been designed.

To achieve this, the in-built char_to_token method has been utilized.

train_answers is a list, each element of that list being a dictionary with 3 keys namely, answer_start, text and answer_end. We iterate over this list and access the start and end positions of each answer using the necessary keys and append the same to two different lists namely start_positions and end_positions respectively.

In [ ]:
train_answers[0]['answer_start']

In [ ]:
train_answers[0]['answer_end']

Since blank space is returned, so we substract 1.

In [ ]:
train_encodings.char_to_token(0,train_answers[0]['answer_end']-1)

In [ ]:
train_encodings.char_to_token(0,train_answers[0]['answer_start'])

Note: 541-515 is dependent on the length of the answer and we can have an idea from this difference as how long was the answer itself.However, the numbers produced via tokenization is just some kind of bijection and apparently nothing to do with the answer length.

In [ ]:
#Refer : https://huggingface.co/transformers/v4.0.1/custom_datasets.html
#Refer: https://huggingface.co/transformers/v4.0.1/main_classes/tokenizer.html#transformers.BatchEncoding.char_to_token


def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
        # if None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)

Note that two more attributes namely 'start_positions' and 'end_positions' got added to the encoding objects.

Each of these is simply a list containing the start and end token positions of the answer that corresponds to the respective question-context pairs.

In [ ]:
train_encodings.keys()

Refer : https://www.tensorflow.org/api_docs/python/tf/data/Dataset

Refer : https://www.tensorflow.org/guide/data

The tf.data.Dataset API supports writing descriptive and efficient input pipelines. Dataset usage follows a common pattern:

Create a source dataset from your input data.
Apply dataset transformations to preprocess the data.
Iterate over the dataset and process the elements.
Iteration happens in a streaming fashion, so the full dataset does not need to fit into memory.

In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    {key: train_encodings[key] for key in ['input_ids', 'attention_mask']},
    {key: train_encodings[key] for key in ['start_positions', 'end_positions']}
))

In [ ]:
from transformers import DistilBertForQuestionAnswering
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")